## Step for make it work

Install Redis for your OS
* pip install redis
* pip install python-redis-rate-limit # https://pypi.org/project/python-redis-rate-limit/
* pip install sparkly https://www.scivision.co/python-windows-visual-c++-14-required/
* pip install sparkly[redis]
* Install http://joeferner.github.io/redis-commander/  
Then `redis-commander --redis-host=localhost --redis-port=6379` from the browser http://localhost:8081/

In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import sys
sys.path.append("..")

In [13]:
from optimus import Optimus
op = Optimus()

In [14]:
# make some test data
columns = ['dogs', 'cats']
vals = [
    (2, 0),
    (0, 1),
    (4, 1)
]

# create DataFrame
df = op.spark.createDataFrame(vals, columns).repartition(1).cache()

In [15]:
df.table()

dogs 1 (bigint),cats 2 (bigint)
2,0
0,1
4,1


## Testing kombu and Rabbitmq

### Activate RabbitMQ GUI Managment
Reference https://www.youtube.com/watch?v=F4PvutsQJlc
Install erlang
Run C:\Program Files\erl10.0.1\Install.exe

Go to C:\Program Files\RabbitMQ Server\rabbitmq_server-3.7.7\sbin

`rabbitmq-plugins enable rabbitmq_management`

`rabbitmq-server.bat restart`

http://localhost:15672/

In [16]:
from kombu import Connection, Exchange, Queue, Consumer, Producer

rabbit_url = "amqp://localhost:5672/"
conn = Connection(rabbit_url)
channel = conn.channel()
exchange = Exchange("example-exchange", type="direct")
queue = Queue(name="example-queue", exchange=exchange, routing_key="BOB")

queue.maybe_bind(conn)
queue.declare()

'example-queue'

In [27]:
# https://medium.com/python-pandemonium/talking-to-rabbitmq-with-python-and-kombu-6cbee93b1298
#https://medium.com/python-pandemonium/building-robust-rabbitmq-consumers-with-python-and-kombu-part-1-ccd660d17271

def func (messages):
    
    channel = conn.channel()    
    producer = Producer(exchange=exchange, channel=channel, routing_key="BOB")
    
    for message in messages:
        as_dict = message.asDict(recursive=True)
        producer.publish(message)
    return messages
        
df.rdd.mapPartitions(func).count()

0

In [ ]:
def process_message(body, message):
    print("The body is {}".format(body))
    message.ack()

    
with Consumer(conn, queues=queue, callbacks=[process_message], accept=["application/json"]):  
    line= conn.drain_events(timeout=5)
    #print("No message in the queue")
    #conn.heartbeat_check()

In [35]:
# https://medium.com/python-pandemonium/building-robust-rabbitmq-consumers-with-python-and-kombu-part-2-e9505f56e12e

from kombu import Connection, Exchange, Queue
from kombu.mixins import ConsumerMixin
rabbit_url = "amqp://localhost:5672/"

class Worker(ConsumerMixin):
    def __init__(self, connection, queues):
        self.connection = connection
        self.queues = queues
    def get_consumers(self, Consumer, channel):
        return [Consumer(queues=self.queues,
                         callbacks=[self.on_message])]
    def on_message(self, body, message):
        print('Got message: {0}'.format(body))
        message.ack()
        
exchange = Exchange("example-exchange", type="direct")
queues = [Queue("example-queue", exchange, routing_key="BOB")]
with Connection(rabbit_url, heartbeat=4) as conn:
        worker = Worker(conn, queues)
        worker.run()



KeyboardInterrupt: 

In [36]:
name, msg_count, consumer_count = queue.queue_declare()

AttributeError: 'list' object has no attribute 'queue_declare'

In [35]:
import socket
import sys

host = '127.0.0.1'
port = 5005


s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((host,port))
s.send(b"1")  
s.close()

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext




ssc  = StreamingContext(op.sc, 1)


lines = ssc.socketTextStream("localhost", 9998)

# Split each line into words
words = lines.flatMap(lambda line: line.split(" "))

# Count each word in each batch
pairs = words.map(lambda word: (word, 1))


wordCounts = pairs.reduceByKey(lambda x, y: x + y)

# Print the first ten elements of each RDD generated in this DStream to the console
wordCounts.pprint()


ssc.start()             # Start the computation
ssc.awaitTermination()  # Wait for the computation to terminate

-------------------------------------------
Time: 2018-09-06 17:28:44
-------------------------------------------

-------------------------------------------
Time: 2018-09-06 17:28:45
-------------------------------------------



In [ ]:
print('joa')